# Reverse Engineering
Este notebook possui o intuito de **tratar os dados** de forma a serem utilizados com **qualidade** na **Análise Exploratória de Dados** e demais atividades relacionadas a este dataset.


### Definição da Base de Dados Relacional:

#### Database: `Produtividade`

- Tabela: `Clientes` (**id_cliente**, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, <u>id_colaborador</u>)

- Tabela: `Colaboradores` (**id_colaborador**, nome, salario_base, horas_semanais, idade, qualificacao, <u>id_equipa</u>)

- Tabela: `Equipas` (**id_equipa**, nome)

- Tabela: `Tarefas` (**id_tarefa**, nome, descricao)

- Tabela: `Rel_Clientes_Colaboradores_Tarefas` (**<u>id_cliente</u>**, **<u>id_colaborador</u>**, **<u>id_tarefa</u>**, data, hora_inicio, hora_fim)

- Tabela: `Clientes` (**id_cliente**, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, <u>id_colaborador</u>)

In [64]:
from databases_connections import *
import pandas as pd
import numpy as np
import json
from time import sleep

### Testando conexões com o banco de dados
- Testando conexão;
- Testando SQL queries;
- Testando criação de tabelas;
- Testando inserção de dados na tabela;

In [36]:
with open("../config/db_config.json") as f:
    config = json.load(f)

print(config)

try:
# instantiating database object
    db = Database(config)
except Exception as e:
    print(e)

{'host': 'localhost', 'port': 3306, 'user': 'root', 'password': 'password', 'database': 'Produtividade'}


In [3]:
db

In [4]:
# testing database connection
try:
    # connecting to database
    db.connect()
    if db.is_connected():
        print("Connection established successfully.")
        
    executor = SQLExecutor(db)

    # closing connection
    db.close()
except Exception as e:
    print(e)

Connection established successfully.


## Queries de criação das tabelas

In [65]:
sql_create_queries = []

- Tabela: `Equipas` (**id_equipa**, nome)

In [66]:
query = """ CREATE TABLE IF NOT EXISTS Equipas (
        id_equipa INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL
        );"""

sql_create_queries.append(query)
print(query)

 CREATE TABLE IF NOT EXISTS Equipas (
        id_equipa INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL
        );


- Tabela: `Colaboradores` (**id_colaborador**, nome, salario_base, horas_semanais, idade, qualificacao, <u>id_equipa</u>)

In [67]:
query = """ CREATE TABLE IF NOT EXISTS Colaboradores (
        id_colaborador INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        salario_base DECIMAL NOT NULL,
        horas_semanais INT NOT NULL,
        idade INT NOT NULL,
        qualificacao VARCHAR(255) NOT NULL,
        id_equipa INT NOT NULL,
        FOREIGN KEY (id_equipa) REFERENCES Equipas(id_equipa)
        );"""

sql_create_queries.append(query)
print(query)

 CREATE TABLE IF NOT EXISTS Colaboradores (
        id_colaborador INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        salario_base DECIMAL NOT NULL,
        horas_semanais INT NOT NULL,
        idade INT NOT NULL,
        qualificacao VARCHAR(255) NOT NULL,
        id_equipa INT NOT NULL,
        FOREIGN KEY (id_equipa) REFERENCES Equipas(id_equipa)
        );


- Tabela: `Clientes` (**id_cliente**, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, <u>id_colaborador</u>)

In [68]:
query = """ CREATE TABLE IF NOT EXISTS Clientes (
        id_cliente INT PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        morada VARCHAR(255),
        distrito VARCHAR(255),
        valor_hora INT NOT NULL,
        horas_mensais_orcamentadas INT NOT NULL,
        satisfacao INT NOT NULL,
        id_colaborador INT NOT NULL,
        FOREIGN KEY (id_colaborador) REFERENCES Colaboradores(id_colaborador)
        ); """

sql_create_queries.append(query)
print(query)

 CREATE TABLE IF NOT EXISTS Clientes (
        id_cliente INT PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        morada VARCHAR(255),
        distrito VARCHAR(255),
        valor_hora INT NOT NULL,
        horas_mensais_orcamentadas INT NOT NULL,
        satisfacao INT NOT NULL,
        id_colaborador INT NOT NULL,
        FOREIGN KEY (id_colaborador) REFERENCES Colaboradores(id_colaborador)
        ); 


- Tabela: `Tarefas` (**id_tarefa**, nome, descricao)

In [69]:
query = """CREATE TABLE IF NOT EXISTS Tarefas (
        id_tarefa INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        descricao VARCHAR(255)
        );"""

sql_create_queries.append(query)
print(query)

CREATE TABLE IF NOT EXISTS Tarefas (
        id_tarefa INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        descricao VARCHAR(255)
        );


- Tabela: `Rel_Clientes_Colaboradores_Tarefas` (**<u>id_cliente</u>**, **<u>id_colaborador</u>**, **<u>id_tarefa</u>**, data, hora_inicio, hora_fim)

In [70]:
query = """CREATE TABLE IF NOT EXISTS Rel_Clientes_Colaboradores_Tarefas (
        id_rel INT NOT NULL,
        id_cliente INT NOT NULL,
        id_colaborador INT NOT NULL,
        id_tarefa INT NOT NULL,
        data_tarefa DATE NOT NULL,
        hora_inicio TIME NOT NULL,
        hora_fim TIME NOT NULL,
        horas DECIMAL(3, 1),
        PRIMARY KEY (id_rel, id_cliente, id_colaborador, id_tarefa),
        FOREIGN KEY (id_cliente) REFERENCES Clientes(id_cliente) ON DELETE CASCADE,
        FOREIGN KEY (id_colaborador) REFERENCES Colaboradores(id_colaborador) ON DELETE CASCADE,
        FOREIGN KEY (id_tarefa) REFERENCES Tarefas(id_tarefa) ON DELETE CASCADE
        );"""

sql_create_queries.append(query)
print(query)

CREATE TABLE IF NOT EXISTS Rel_Clientes_Colaboradores_Tarefas (
        id_rel INT NOT NULL,
        id_cliente INT NOT NULL,
        id_colaborador INT NOT NULL,
        id_tarefa INT NOT NULL,
        data_tarefa DATE NOT NULL,
        hora_inicio TIME NOT NULL,
        hora_fim TIME NOT NULL,
        horas DECIMAL(3, 1),
        PRIMARY KEY (id_rel, id_cliente, id_colaborador, id_tarefa),
        FOREIGN KEY (id_cliente) REFERENCES Clientes(id_cliente) ON DELETE CASCADE,
        FOREIGN KEY (id_colaborador) REFERENCES Colaboradores(id_colaborador) ON DELETE CASCADE,
        FOREIGN KEY (id_tarefa) REFERENCES Tarefas(id_tarefa) ON DELETE CASCADE
        );


In [71]:
for query in sql_create_queries:
    print(query)

 CREATE TABLE IF NOT EXISTS Equipas (
        id_equipa INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL
        );
 CREATE TABLE IF NOT EXISTS Colaboradores (
        id_colaborador INT NOT NULL PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        salario_base DECIMAL NOT NULL,
        horas_semanais INT NOT NULL,
        idade INT NOT NULL,
        qualificacao VARCHAR(255) NOT NULL,
        id_equipa INT NOT NULL,
        FOREIGN KEY (id_equipa) REFERENCES Equipas(id_equipa)
        );
 CREATE TABLE IF NOT EXISTS Clientes (
        id_cliente INT PRIMARY KEY,
        nome VARCHAR(255) NOT NULL,
        morada VARCHAR(255),
        distrito VARCHAR(255),
        valor_hora INT NOT NULL,
        horas_mensais_orcamentadas INT NOT NULL,
        satisfacao INT NOT NULL,
        id_colaborador INT NOT NULL,
        FOREIGN KEY (id_colaborador) REFERENCES Colaboradores(id_colaborador)
        ); 
CREATE TABLE IF NOT EXISTS Tarefas (
        id_tarefa INT NOT NULL PRIMARY K

## Executando queries de criação das tabelas

In [169]:
try:
    db.connect()
    executor = SQLExecutor(db)
    
    executor.execute("BEGIN")
    
    for query in sql_create_queries:
        executor.execute(query)
        sleep(0.1)
    
    executor.execute("COMMIT")
    
    db.close()
except Exception as e:
    print(e)
    if db.is_connected():
        db.close()

## Reverse Engineering

In [72]:
df_tempos = pd.read_excel('../data/produtividade.xlsx', sheet_name='RegistoTempos')
df_tempos.head()

,Data,Hora Início,Hora Fim,Horas,Colaborador,Tarefa,Cliente
0,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 5,Orçamento Anual,Cliente 19
1,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 7,Preenchimento de Documentos,Cliente 13
2,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 2,Lançamento Documentos,Cliente 7
3,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 8,Reuniões de Equipa,Cliente 24
4,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 2,Reuniões de Equipa,Cliente 8


In [73]:
df_tarefas = pd.read_excel('../data/produtividade.xlsx', sheet_name='Tarefas')
df_tarefas.head()

,COD_Tarefa,Tarefa,Descrição
0,1,Lançamento Documentos,NaN
1,2,Validação,NaN
2,3,Gestão Comercial,NaN
3,4,Reuniões de Equipa,NaN
4,5,Reuniões de Coordenação,NaN


In [74]:
df_clientes = pd.read_excel('../data/produtividade.xlsx', sheet_name='Clientes')
df_clientes.head()

,Cod_Cliente,Cliente,Morada,Distrito,Colaborador,Valor Hora,Horas Mensais Orçamentadas,Satisfação Cliente (1-10)
0,1,Cliente 1,NaN,NaN,Colaborador 1,50,38,8
1,2,Cliente 2,NaN,NaN,Colaborador 6,50,21,7
2,3,Cliente 3,NaN,NaN,Colaborador 7,50,24,10
3,4,Cliente 4,NaN,NaN,Colaborador 2,50,118,8
4,5,Cliente 5,NaN,NaN,Colaborador 9,50,39,7


In [75]:
df_colaboradores = pd.read_excel('../data/produtividade.xlsx', sheet_name='Colaboradores')
df_colaboradores.head()

,Nome,Salário Base,Horas Semana,Equipa,Idade,Qualificações
0,Colaborador 1,850,40,SEDE,23,Licenciatura
1,Colaborador 2,875,40,SEDE,25,Mestrado
2,Colaborador 3,800,40,SEDE,32,Mestrado
3,Colaborador 4,900,40,FARO,21,Licenciatura
4,Colaborador 5,1000,40,PORTIMÃO,35,Mestrado


In [76]:
df_colaboradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nome           9 non-null      object
 1   Salário Base   9 non-null      int64 
 2   Horas Semana   9 non-null      int64 
 3   Equipa         9 non-null      object
 4   Idade          9 non-null      int64 
 5   Qualificações  9 non-null      object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [77]:
df_tarefas = df_tarefas.rename(columns={'COD_Tarefa': 'id_tarefa', 'Tarefa': 'nome', 'Descrição': 'descricao'})
df_tarefas.head()

,id_tarefa,nome,descricao
0,1,Lançamento Documentos,NaN
1,2,Validação,NaN
2,3,Gestão Comercial,NaN
3,4,Reuniões de Equipa,NaN
4,5,Reuniões de Coordenação,NaN


In [78]:
df_tarefas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_tarefa  15 non-null     int64  
 1   nome       15 non-null     object 
 2   descricao  0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 488.0+ bytes


In [79]:
sql_insert_queries = []

In [80]:
sql_insert_tarefas = []
dic_tarefas = {}
for index, row in df_tarefas.iterrows():
    sql_insert_tarefas.append(f"""INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES ({row['id_tarefa']}, '{row['nome']}', '{row['descricao']}');""")
    dic_tarefas[row['nome']] = row['id_tarefa']
    
print(*sql_insert_tarefas, sep='\n')
dic_tarefas

INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (1, 'Lançamento Documentos', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (2, 'Validação', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (3, 'Gestão Comercial', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (4, 'Reuniões de Equipa', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (5, 'Reuniões de Coordenação', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (6, 'Preenchimento de Documentos', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (7, 'Fecho do Ano', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (8, 'Fecho do Mês', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (9, 'Declarações do IVA', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (10, 'Processamento de Salários', 'nan');
INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (11, 'Orçamento Anual', 'nan');
INSERT INT

{'Lançamento Documentos': 1,
 'Validação': 2,
 'Gestão Comercial': 3,
 'Reuniões de Equipa': 4,
 'Reuniões de Coordenação': 5,
 'Preenchimento de Documentos': 6,
 'Fecho do Ano': 7,
 'Fecho do Mês': 8,
 'Declarações do IVA': 9,
 'Processamento de Salários': 10,
 'Orçamento Anual': 11,
 'Processos Recrutamento': 12,
 'Reuniões Clientes': 13,
 'Atendimento Telf. / Email': 14,
 'Contactos Estado': 15}

- Tabela: `Colaboradores` (**id_colaborador**, nome, salario_base, horas_semanais, idade, qualificacao, <u>id_equipa</u>)

In [81]:
df_colaboradores.columns

Index(['Nome', 'Salário Base', 'Horas Semana', 'Equipa', 'Idade',
       'Qualificações'],
      dtype='object')

In [82]:
df_colaboradores = df_colaboradores.rename(columns={'Nome': 'nome', 'Salário Base': 'salario_base', 'Horas Semana': 'horas_semanais', 'Idade': 'idade', 'Qualificações': 'qualificacao', 'Equipa': 'equipa'})
df_colaboradores.head()

,nome,salario_base,horas_semanais,equipa,idade,qualificacao
0,Colaborador 1,850,40,SEDE,23,Licenciatura
1,Colaborador 2,875,40,SEDE,25,Mestrado
2,Colaborador 3,800,40,SEDE,32,Mestrado
3,Colaborador 4,900,40,FARO,21,Licenciatura
4,Colaborador 5,1000,40,PORTIMÃO,35,Mestrado


In [83]:
df_colaboradores.equipa.unique()

array(['SEDE', 'FARO', 'PORTIMÃO', 'LAGOS'], dtype=object)

In [84]:
sql_insert_equipas = []
dic_equipas = {}

for index, row in enumerate(df_colaboradores.equipa.unique(), start=1):
    sql_insert_equipas.append(f"""INSERT INTO Equipas (id_equipa, nome) VALUES ({index}, '{row}');""")
    dic_equipas[row] = index

print(*sql_insert_equipas, sep='\n')
dic_equipas

INSERT INTO Equipas (id_equipa, nome) VALUES (1, 'SEDE');
INSERT INTO Equipas (id_equipa, nome) VALUES (2, 'FARO');
INSERT INTO Equipas (id_equipa, nome) VALUES (3, 'PORTIMÃO');
INSERT INTO Equipas (id_equipa, nome) VALUES (4, 'LAGOS');


{'SEDE': 1, 'FARO': 2, 'PORTIMÃO': 3, 'LAGOS': 4}

Encoding `equipas` de acordo com o `id_equipa`:

In [85]:
df_colaboradores['id_equipa'] = df_colaboradores['equipa'].map(dic_equipas)
df_colaboradores.drop(columns=['equipa'], inplace=True)
df_colaboradores.head()

,nome,salario_base,horas_semanais,idade,qualificacao,id_equipa
0,Colaborador 1,850,40,23,Licenciatura,1
1,Colaborador 2,875,40,25,Mestrado,1
2,Colaborador 3,800,40,32,Mestrado,1
3,Colaborador 4,900,40,21,Licenciatura,2
4,Colaborador 5,1000,40,35,Mestrado,3


In [86]:
df_colaboradores.nome.unique()

array(['Colaborador 1', 'Colaborador 2', 'Colaborador 3', 'Colaborador 4',
       'Colaborador 5', 'Colaborador 6', 'Colaborador 7', 'Colaborador 8',
       'Colaborador 9'], dtype=object)

In [87]:
dic_colaboradores = {}
for index, row in enumerate(df_colaboradores.nome.unique(), start=1):
    dic_colaboradores[row] = index

dic_colaboradores

{'Colaborador 1': 1,
 'Colaborador 2': 2,
 'Colaborador 3': 3,
 'Colaborador 4': 4,
 'Colaborador 5': 5,
 'Colaborador 6': 6,
 'Colaborador 7': 7,
 'Colaborador 8': 8,
 'Colaborador 9': 9}

In [88]:
df_colaboradores['id_colaborador'] = df_colaboradores['nome'].map(dic_colaboradores)
df_colaboradores.head()

,nome,salario_base,horas_semanais,idade,qualificacao,id_equipa,id_colaborador
0,Colaborador 1,850,40,23,Licenciatura,1,1
1,Colaborador 2,875,40,25,Mestrado,1,2
2,Colaborador 3,800,40,32,Mestrado,1,3
3,Colaborador 4,900,40,21,Licenciatura,2,4
4,Colaborador 5,1000,40,35,Mestrado,3,5


In [89]:
sql_insert_colaboradores = []
for index, row in df_colaboradores.iterrows():
    sql_insert_colaboradores.append(f"""INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES ({row['id_colaborador']}, '{row['nome']}', {row['salario_base']}, {row['horas_semanais']}, {row['idade']}, '{row['qualificacao']}', {row['id_equipa']});""")

sql_insert_colaboradores

["INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES (1, 'Colaborador 1', 850, 40, 23, 'Licenciatura', 1);",
 "INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES (2, 'Colaborador 2', 875, 40, 25, 'Mestrado', 1);",
 "INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES (3, 'Colaborador 3', 800, 40, 32, 'Mestrado', 1);",
 "INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES (4, 'Colaborador 4', 900, 40, 21, 'Licenciatura', 2);",
 "INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES (5, 'Colaborador 5', 1000, 40, 35, 'Mestrado', 3);",
 "INSERT INTO Colaboradores (id_colaborador, nome, salario_base, horas_semanais, idade, qualificacao, id_equipa) VALUES (6, 'Colabo

In [90]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Cod_Cliente                 25 non-null     int64  
 1   Cliente                     25 non-null     object 
 2   Morada                      0 non-null      float64
 3   Distrito                    0 non-null      float64
 4   Colaborador                 25 non-null     object 
 5   Valor Hora                  25 non-null     int64  
 6   Horas Mensais Orçamentadas  25 non-null     int64  
 7   Satisfação Cliente (1-10)   25 non-null     int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.7+ KB


Casting `Morada` e `Distrito` para **str**

In [91]:
df_clientes['Morada'] = df_clientes['Morada'].astype(str)
df_clientes['Distrito'] = df_clientes['Distrito'].astype(str)
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Cod_Cliente                 25 non-null     int64 
 1   Cliente                     25 non-null     object
 2   Morada                      25 non-null     object
 3   Distrito                    25 non-null     object
 4   Colaborador                 25 non-null     object
 5   Valor Hora                  25 non-null     int64 
 6   Horas Mensais Orçamentadas  25 non-null     int64 
 7   Satisfação Cliente (1-10)   25 non-null     int64 
dtypes: int64(4), object(4)
memory usage: 1.7+ KB


- Tabela: `Clientes` (**id_cliente**, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, <u>id_colaborador</u>)

In [92]:
df_clientes = df_clientes.rename(columns={'Cod_Cliente':'id_cliente', 'Cliente': 'nome', 'Morada': 'morada', 'Distrito': 'distrito', 'Valor Hora': 'valor_hora', 'Horas Mensais Orçamentadas': 'horas_mensais_orcamentadas', 'Satisfação Cliente (1-10)': 'satisfacao', 'Colaborador': 'colaborador'})
df_clientes.columns

Index(['id_cliente', 'nome', 'morada', 'distrito', 'colaborador', 'valor_hora',
       'horas_mensais_orcamentadas', 'satisfacao'],
      dtype='object')

In [93]:
df_clientes['colaborador'] = df_clientes['colaborador'].map(dic_colaboradores)
df_clientes.rename(columns={'colaborador': 'id_colaborador'}, inplace=True)
df_clientes.head()

,id_cliente,nome,morada,distrito,id_colaborador,valor_hora,horas_mensais_orcamentadas,satisfacao
0,1,Cliente 1,nan,nan,1,50,38,8
1,2,Cliente 2,nan,nan,6,50,21,7
2,3,Cliente 3,nan,nan,7,50,24,10
3,4,Cliente 4,nan,nan,2,50,118,8
4,5,Cliente 5,nan,nan,9,50,39,7


In [94]:
sql_insert_clientes = []
dic_clientes = {}
for index, row in df_clientes.iterrows():
    sql_insert_clientes.append(f"""INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, id_colaborador) VALUES ({row['id_cliente']}, '{row['nome']}', '{row['morada']}', '{row['distrito']}', {row['valor_hora']}, {row['horas_mensais_orcamentadas']}, {row['satisfacao']}, {row['id_colaborador']});""")
    dic_clientes[row['nome']] = row['id_cliente']

print(*sql_insert_clientes, sep='\n')
dic_clientes

INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, id_colaborador) VALUES (1, 'Cliente 1', 'nan', 'nan', 50, 38, 8, 1);
INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, id_colaborador) VALUES (2, 'Cliente 2', 'nan', 'nan', 50, 21, 7, 6);
INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, id_colaborador) VALUES (3, 'Cliente 3', 'nan', 'nan', 50, 24, 10, 7);
INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, id_colaborador) VALUES (4, 'Cliente 4', 'nan', 'nan', 50, 118, 8, 2);
INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satisfacao, id_colaborador) VALUES (5, 'Cliente 5', 'nan', 'nan', 50, 39, 7, 9);
INSERT INTO Clientes (id_cliente, nome, morada, distrito, valor_hora, horas_mensais_orcamentadas, satis

{'Cliente 1': 1,
 'Cliente 2': 2,
 'Cliente 3': 3,
 'Cliente 4': 4,
 'Cliente 5': 5,
 'Cliente 6': 6,
 'Cliente 7': 7,
 'Cliente 8': 8,
 'Cliente 9': 9,
 'Cliente 10': 10,
 'Cliente 11': 11,
 'Cliente 12': 12,
 'Cliente 13': 13,
 'Cliente 14': 14,
 'Cliente 15': 15,
 'Cliente 16': 16,
 'Cliente 17': 17,
 'Cliente 18': 18,
 'Cliente 19': 19,
 'Cliente 20': 20,
 'Cliente 21': 21,
 'Cliente 22': 22,
 'Cliente 23': 23,
 'Cliente 24': 24,
 'Cliente 25': 25}

- Tabela: `Rel_Clientes_Colaboradores_Tarefas` (**<u>id_cliente</u>**, **<u>id_colaborador</u>**, **<u>id_tarefa</u>**, data, hora_inicio, hora_fim, horas)

In [95]:
df_tempos

,Data,Hora Início,Hora Fim,Horas,Colaborador,Tarefa,Cliente
0,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 5,Orçamento Anual,Cliente 19
1,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 7,Preenchimento de Documentos,Cliente 13
2,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 2,Lançamento Documentos,Cliente 7
3,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 8,Reuniões de Equipa,Cliente 24
4,2021-01-02,00:00:00,00:00:00,0.0,Colaborador 2,Reuniões de Equipa,Cliente 8
...,...,...,...,...,...,...,...
52411,2021-12-31,14:30:00,15:00:00,0.5,Colaborador 4,Fecho do Ano,Cliente 11
52412,2021-12-31,15:00:00,15:30:00,0.5,Colaborador 4,Declarações do IVA,Cliente 14
52413,2021-12-31,15:30:00,16:00:00,0.5,Colaborador 7,Atendimento Telf. / Email,Cliente 13
52414,2021-12-31,16:00:00,16:30:00,0.5,Colaborador 9,Atendimento Telf. / Email,Cliente 25


In [96]:
df_tempos['id_rel'] = df_tempos.index + 1

In [97]:
df_tempos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52416 entries, 0 to 52415
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Data          52416 non-null  datetime64[ns]
 1   Hora Início   52416 non-null  object        
 2   Hora Fim      52416 non-null  object        
 3   Horas         52416 non-null  float64       
 4   Colaborador   52416 non-null  object        
 5   Tarefa        52416 non-null  object        
 6   Cliente       52416 non-null  object        
 7   id_rel        52416 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 3.2+ MB


In [98]:
df_tempos['Colaborador'] = df_tempos['Colaborador'].map(dic_colaboradores)
df_tempos.rename(columns={'Colaborador': 'id_colaborador'}, inplace=True)
df_tempos['Cliente'] = df_tempos['Cliente'].map(dic_clientes)
df_tempos.rename(columns={'Cliente': 'id_cliente'}, inplace=True)
df_tempos['Tarefa'] = df_tempos['Tarefa'].map(dic_tarefas)
df_tempos.rename(columns={'Tarefa': 'id_tarefa'}, inplace=True)
df_tempos.head()

,Data,Hora Início,Hora Fim,Horas,id_colaborador,id_tarefa,id_cliente,id_rel
0,2021-01-02,00:00:00,00:00:00,0.0,5,11,19,1
1,2021-01-02,00:00:00,00:00:00,0.0,7,6,13,2
2,2021-01-02,00:00:00,00:00:00,0.0,2,1,7,3
3,2021-01-02,00:00:00,00:00:00,0.0,8,4,24,4
4,2021-01-02,00:00:00,00:00:00,0.0,2,4,8,5


In [99]:
df_tempos.columns

Index(['Data', 'Hora Início ', 'Hora Fim', 'Horas', 'id_colaborador',
       'id_tarefa', 'id_cliente', 'id_rel'],
      dtype='object')

In [100]:
df_tempos.rename(columns={'Data': 'data_tarefa', 'Hora Início ': 'hora_inicio', 'Hora Fim': 'hora_fim', 'Horas': 'horas'}, inplace=True)
df_tempos.head()

,data_tarefa,hora_inicio,hora_fim,horas,id_colaborador,id_tarefa,id_cliente,id_rel
0,2021-01-02,00:00:00,00:00:00,0.0,5,11,19,1
1,2021-01-02,00:00:00,00:00:00,0.0,7,6,13,2
2,2021-01-02,00:00:00,00:00:00,0.0,2,1,7,3
3,2021-01-02,00:00:00,00:00:00,0.0,8,4,24,4
4,2021-01-02,00:00:00,00:00:00,0.0,2,4,8,5


In [101]:
sql_insert_rel = []
for index, row in df_tempos.iterrows():
    sql_insert_rel.append(f"""INSERT INTO Rel_Clientes_Colaboradores_Tarefas (id_rel, data_tarefa, hora_inicio, hora_fim, horas, id_colaborador, id_cliente, id_tarefa) VALUES ({row['id_rel']}, '{row['data_tarefa']}', '{row['hora_inicio']}', '{row['hora_fim']}', {row['horas']}, {row['id_colaborador']}, {row['id_cliente']}, {row['id_tarefa']});""")

sql_insert_rel[0:5]

["INSERT INTO Rel_Clientes_Colaboradores_Tarefas (id_rel, data_tarefa, hora_inicio, hora_fim, horas, id_colaborador, id_cliente, id_tarefa) VALUES (1, '2021-01-02 00:00:00', '00:00:00', '00:00:00', 0.0, 5, 19, 11);",
 "INSERT INTO Rel_Clientes_Colaboradores_Tarefas (id_rel, data_tarefa, hora_inicio, hora_fim, horas, id_colaborador, id_cliente, id_tarefa) VALUES (2, '2021-01-02 00:00:00', '00:00:00', '00:00:00', 0.0, 7, 13, 6);",
 "INSERT INTO Rel_Clientes_Colaboradores_Tarefas (id_rel, data_tarefa, hora_inicio, hora_fim, horas, id_colaborador, id_cliente, id_tarefa) VALUES (3, '2021-01-02 00:00:00', '00:00:00', '00:00:00', 0.0, 2, 7, 1);",
 "INSERT INTO Rel_Clientes_Colaboradores_Tarefas (id_rel, data_tarefa, hora_inicio, hora_fim, horas, id_colaborador, id_cliente, id_tarefa) VALUES (4, '2021-01-02 00:00:00', '00:00:00', '00:00:00', 0.0, 8, 24, 4);",
 "INSERT INTO Rel_Clientes_Colaboradores_Tarefas (id_rel, data_tarefa, hora_inicio, hora_fim, horas, id_colaborador, id_cliente, id_tare

In [102]:
if (len(sql_insert_rel) == len(df_tempos)):
    print('Quantidade de queries consistente com quantidade de registos!')
else:
    print('Quantidade de queries inconsistente com quantidade de registos!')

Quantidade de queries consistente com quantidade de registos!


In [103]:
sql_insert_queries = sql_insert_tarefas + sql_insert_equipas + sql_insert_colaboradores + sql_insert_clientes + sql_insert_rel
sql_insert_queries[0:5]

["INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (1, 'Lançamento Documentos', 'nan');",
 "INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (2, 'Validação', 'nan');",
 "INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (3, 'Gestão Comercial', 'nan');",
 "INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (4, 'Reuniões de Equipa', 'nan');",
 "INSERT INTO Tarefas (id_tarefa, nome, descricao) VALUES (5, 'Reuniões de Coordenação', 'nan');"]

In [104]:
if len(sql_insert_queries) == len(sql_insert_tarefas) + len(sql_insert_equipas) + len(sql_insert_colaboradores) + len(sql_insert_clientes) + len(sql_insert_rel):
    print('Quantidade de queries consistente com quantidade de registos!')
else:
    print('Quantidade de queries inconsistente com quantidade de registos!')

Quantidade de queries consistente com quantidade de registos!


## Save de SQL queries

In [120]:
tabelas = {'tarefas': sql_insert_tarefas,
          'equipas': sql_insert_equipas,
          'colaboradores': sql_insert_colaboradores,
          'clientes': sql_insert_clientes,
          'rel': sql_insert_rel}

with open('../sql/sql_create_queries.sql', 'w') as f:
    f.write('\n'.join(sql_create_queries))

for key, value in tabelas.items():
    with open(f"../sql/sql_insert_{key}.sql", 'w') as f:
        f.write('\n'.join(value))

## Executando queries de inserção de dados nas tabelas

In [ ]:
try:
    db.connect()
    executor = SQLExecutor(db)
    
    executor.execute("BEGIN")
    
    for query in sql_insert_queries:
        executor.execute(query)
        sleep(0.1)
    
    executor.execute("COMMIT")
    
    db.close()
except Exception as e:
    print(e)
    if db.is_connected():
        db.close()

## Criando dataset a partir da base de dados relacional

In [5]:
def get_frame_from_table(table: str, db, executor) -> pd.DataFrame:
    """ Get DataFrame from table in database """
    db.connect()
    executor = SQLExecutor(db)
    executor.execute("BEGIN")
    results = executor.execute(f"SELECT * FROM {table};")
    columns = executor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table}';")
    executor.execute("COMMIT")
    db.close()
    return pd.DataFrame(results, columns=[column[0] for column in columns])

In [51]:
df = get_frame_from_table('Rel_Clientes_Colaboradores_Tarefas', db, executor)
df

,id_rel,id_cliente,id_colaborador,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas
0,1,19,5,11,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0
1,2,13,7,6,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0
2,3,7,2,1,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0
3,4,24,8,4,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0
4,5,8,2,4,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0
...,...,...,...,...,...,...,...,...
52411,52412,11,4,7,2021-12-31,0 days 14:30:00,0 days 15:00:00,0.5
52412,52413,14,4,9,2021-12-31,0 days 15:00:00,0 days 15:30:00,0.5
52413,52414,13,7,14,2021-12-31,0 days 15:30:00,0 days 16:00:00,0.5
52414,52415,25,9,14,2021-12-31,0 days 16:00:00,0 days 16:30:00,0.5


In [52]:
merged = pd.merge(df, get_frame_from_table('Colaboradores', db, executor), on='id_colaborador').rename(columns={'nome': 'colaborador'})
merged.head()

,id_rel,id_cliente,id_colaborador,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas,colaborador,salario_base,horas_semanais,idade,qualificacao,id_equipa
0,1,19,5,11,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3
1,8,19,5,8,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3
2,9,19,5,6,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3
3,20,19,5,14,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3
4,25,19,5,15,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3


In [53]:
merged = pd.merge(merged, get_frame_from_table('Equipas', db, executor), on='id_equipa', how='left').rename(columns={'nome': 'equipa'})
merged.head()

,id_rel,id_cliente,id_colaborador,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas,colaborador,salario_base,horas_semanais,idade,qualificacao,id_equipa,equipa
0,1,19,5,11,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO
1,8,19,5,8,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO
2,9,19,5,6,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO
3,20,19,5,14,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO
4,25,19,5,15,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO


In [54]:
merged = pd.merge(merged, get_frame_from_table('Clientes', db, executor), on='id_cliente').rename(columns={'nome': 'cliente'})
merged.head()

,id_rel,id_cliente,id_colaborador_x,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas,colaborador,salario_base,...,qualificacao,id_equipa,equipa,cliente,morada,distrito,valor_hora,horas_mensais_orcamentadas,satisfacao,id_colaborador_y
0,1,19,5,11,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,...,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,5
1,8,19,5,8,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,...,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,5
2,9,19,5,6,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,...,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,5
3,20,19,5,14,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,...,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,5
4,25,19,5,15,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,...,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,5


In [55]:
merged = merged.rename(columns={'id_colaborador_x': 'id_colaborador'}).drop(columns=['id_colaborador_y'])
merged.head()

,id_rel,id_cliente,id_colaborador,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas,colaborador,salario_base,...,idade,qualificacao,id_equipa,equipa,cliente,morada,distrito,valor_hora,horas_mensais_orcamentadas,satisfacao
0,1,19,5,11,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,...,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8
1,8,19,5,8,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,...,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8
2,9,19,5,6,2021-01-02,0 days,0 days,0.0,Colaborador 5,1000,...,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8
3,20,19,5,14,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,...,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8
4,25,19,5,15,2021-01-03,0 days,0 days,0.0,Colaborador 5,1000,...,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8


In [56]:
merged = pd.merge(merged, get_frame_from_table('Tarefas', db, executor), on='id_tarefa').rename(columns={'nome': 'tarefa'})
merged.head()

,id_rel,id_cliente,id_colaborador,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas,colaborador,salario_base,...,id_equipa,equipa,cliente,morada,distrito,valor_hora,horas_mensais_orcamentadas,satisfacao,tarefa,descricao
0,1,19,5,11,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0,Colaborador 5,1000,...,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
1,192,19,5,11,2021-01-13,0 days 16:30:00,0 days 17:00:00,0.5,Colaborador 5,1000,...,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
2,307,19,5,11,2021-01-21,0 days 10:00:00,0 days 10:30:00,0.5,Colaborador 5,1000,...,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
3,465,19,5,11,2021-01-31,0 days 00:00:00,0 days 00:00:00,0.0,Colaborador 5,1000,...,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
4,468,19,5,11,2021-01-31,0 days 00:00:00,0 days 00:00:00,0.0,Colaborador 5,1000,...,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan


In [57]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52416 entries, 0 to 52415
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   id_rel                      52416 non-null  int64          
 1   id_cliente                  52416 non-null  int64          
 2   id_colaborador              52416 non-null  int64          
 3   id_tarefa                   52416 non-null  int64          
 4   data_tarefa                 52416 non-null  object         
 5   hora_inicio                 52416 non-null  timedelta64[ns]
 6   hora_fim                    52416 non-null  timedelta64[ns]
 7   horas                       52416 non-null  object         
 8   colaborador                 52416 non-null  object         
 9   salario_base                52416 non-null  object         
 10  horas_semanais              52416 non-null  int64          
 11  idade                       52416 non-nul

In [58]:
merged['data_tarefa'] = pd.to_datetime(merged['data_tarefa'])
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52416 entries, 0 to 52415
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   id_rel                      52416 non-null  int64          
 1   id_cliente                  52416 non-null  int64          
 2   id_colaborador              52416 non-null  int64          
 3   id_tarefa                   52416 non-null  int64          
 4   data_tarefa                 52416 non-null  datetime64[ns] 
 5   hora_inicio                 52416 non-null  timedelta64[ns]
 6   hora_fim                    52416 non-null  timedelta64[ns]
 7   horas                       52416 non-null  object         
 8   colaborador                 52416 non-null  object         
 9   salario_base                52416 non-null  object         
 10  horas_semanais              52416 non-null  int64          
 11  idade                       52416 non-nul

## Save dataset

In [59]:
with open('../data/dataset.csv', 'w') as f:
    merged.astype(merged.dtypes.to_dict()).to_csv(f, index=False)

In [60]:
# Save the dtypes as a dictionary in a JSON file

dtype_mapping = {col: str(dtype) for col, dtype in merged.dtypes.items()}

with open('../config/dtypes.json', 'w') as f:
    json.dump(dtype_mapping, f)

In [61]:
# cat ../data/dataset.csv first 10 lines
!head ../data/dataset.csv

id_rel,id_cliente,id_colaborador,id_tarefa,data_tarefa,hora_inicio,hora_fim,horas,colaborador,salario_base,horas_semanais,idade,qualificacao,id_equipa,equipa,cliente,morada,distrito,valor_hora,horas_mensais_orcamentadas,satisfacao,tarefa,descricao
1,19,5,11,2021-01-02,0 days 00:00:00,0 days 00:00:00,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
192,19,5,11,2021-01-13,0 days 16:30:00,0 days 17:00:00,0.5,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
307,19,5,11,2021-01-21,0 days 10:00:00,0 days 10:30:00,0.5,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
465,19,5,11,2021-01-31,0 days 00:00:00,0 days 00:00:00,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anual,nan
468,19,5,11,2021-01-31,0 days 00:00:00,0 days 00:00:00,0.0,Colaborador 5,1000,40,35,Mestrado,3,PORTIMÃO,Cliente 19,nan,nan,50,186,8,Orçamento Anua

In [62]:
!head ../data/dtypes.json

{"id_rel": "int64", "id_cliente": "int64", "id_colaborador": "int64", "id_tarefa": "int64", "data_tarefa": "datetime64[ns]", "hora_inicio": "timedelta64[ns]", "hora_fim": "timedelta64[ns]", "horas": "object", "colaborador": "object", "salario_base": "object", "horas_semanais": "int64", "idade": "int64", "qualificacao": "object", "id_equipa": "int64", "equipa": "object", "cliente": "object", "morada": "object", "distrito": "object", "valor_hora": "int64", "horas_mensais_orcamentadas": "int64", "satisfacao": "int64", "tarefa": "object", "descricao": "object"}